In [17]:
import numpy as np
from cmath import exp

class ConvNeuralNetwork():
    def __init__(self) -> None:
        self.filters = []

    def add_conv_layer(self,num_filters: int,filter_size: tuple):
        h,w = filter_size
        filter = [[np.random.randn() for _ in range(w)] for _ in range(h)]
        self.filters.append({'type': 'conv', 'num_filters': num_filters,'size': filter_size, 'filter': filter})
    
    def add_maxpool_layer(self,size:tuple):
        self.filters.append({'type': 'maxpool','size': size})
    
    def add_fully_connected_layer(self,input_size: int, ouput_size: int, activation_function = lambda x : 1 / (1 + exp(-x).real)):
        weights = [[np.random.randn() for _ in range(input_size)] for _ in range(ouput_size)]
        self.filters.append({'type': 'fully connected','weights':weights, 'activation_function': activation_function})
    
    def _apply_conv_layer(self,input, layer: dict):
        width, height, depth = len(input[-1]), len(input), len(input[-1][-1])
        filter = layer['filter']
        filter_size = layer['size']
        num_filters = layer['num_filters']

        start_i = filter_size[0] // 2
        final_i = height - filter_size[0] // 2
        start_j = filter_size[1] // 2
        final_j = width - filter_size[1] // 2

        result = []
        for i in range(start_i, final_i):
            result.append([])
            for j in range(start_j, final_j):
                result[-1].append([])
                for _ in range(num_filters):
                    for k in range(depth):
                        s = 0
                        for h in range(filter_size[0]):
                            for w in range(filter_size[1]):
                                s += filter[h][w]*input[i+h-filter_size[0]//2][j+w-filter_size[1]//2][k]
                        
                        result[i-start_i][j-start_j].append(s)
        
        return result

    def _apply_maxpool_layer(self, input, layer):
        h, w = layer['size']
        width, height, depth = len(input[-1]), len(input), len(input[-1][-1])
        result = []
        for i in range(0, height - h + 1, h):
            result.append([])
            for j in range(0, width - w + 1, w):
                result[-1].append([])
                for k in range(depth):
                    values = []
                    for row in range(i,i+h):
                        for col in range(j,j+w):
                            values.append(input[row][col][k])
                    result[-1][-1].append(max(values))
        
        return result

    def _linear_input(self, input):
        if all(isinstance(x, (int,float)) for x in input):
            return input
        result = []
        width, height, depth = len(input[-1]), len(input), len(input[-1][-1])
        for i in range(height):
            for j in range(width):
                for k in range(depth):
                    result.append(input[i][j][k])
        return result

    def _apply_fully_connected_layer(self, input, layer):
        lin_input = self._linear_input(input)
        weights = layer['weights']
        sigmoid = layer['activation_function']
        results = []
        for w in weights:
            val = sum([w[i] * lin_input[i] for i in range(len(w))])
            s = sigmoid(val)
            results.append(s)
        return results

    def _apply_layer(self, input, layer):
        type = layer['type']
        if type == 'conv':
            return self._apply_conv_layer(input, layer)
        elif type == 'maxpool':
            return self._apply_maxpool_layer(input, layer)
        elif type == 'fully connected':
            return self._apply_fully_connected_layer(input, layer)

    def _back_propagation_fully_connected_layer(self,layer,errors, input_line):
        learning_rate = 0.00000001
        for i in range(0,len(errors)):
            err = errors[i]
            r = input_line
            for l in self.filters[:-1]:
                r = self._apply_layer(r,l)
            r = self._linear_input(r)
            for j in range(len(r)):
                layer['weights'][i][j] = layer['weights'][i][j] - learning_rate * err * r[j]

    def _back_propagation_conv_layer(self, layer, errors):
        learning_rate = 0.00000001
        for i in range(0, len(errors)):
            filter = layer['filter']
            for k1 in range(len(filter)):
                for k2 in range(len(filter[k1])):
                    layer['filter'][k1][k2] = layer['filter'][k1][k2] - errors[i]*learning_rate
            
    def _back_propagation(self, errors, input_line):
        for layer in reversed(self.filters):
            type = layer['type']
            if type == 'fully connected':
                self._back_propagation_fully_connected_layer(layer, errors, input_line)
            if type == 'conv':
                self._back_propagation_conv_layer(layer,errors)

    def _apply_layers(self, input_line):
        result = input_line
        for layer in self.filters:
            result = self._apply_layer(result, layer)
        return result

    def train(self, input, output, epoch):
        for e in range(epoch):
            loss = 0
            for i in range(len(input)):
                input_line = input[i]
                o = output[i]
                result = self._apply_layers(input_line)
                yes_prob = result[0]
                no_prob = result[1]
                if o == 'YES':
                    D = 1
                    err = [1-yes_prob, 0-no_prob]
                else:
                    D = 0
                    err = [0-yes_prob, 1-no_prob]
                
                loss = loss + abs(D*np.log(yes_prob) + (1-D)*np.log(1-yes_prob))
                
                self._back_propagation(err,input_line)
            loss = loss/len(input)
            print(f"Epoch {e} -------> loss: {loss}")

    def predict(self,input):
        output = []
        for input_line in input:
            result = self._apply_layers(input_line)
            if result[0] > result[1]:
                output.append('YES')
            else:
                output.append('NO')
        return output


In [18]:
def get_TP_TN_FP_FN(truth,computed,positive_label):
    TP = TN = FP = FN = 0
    for i in range(len(truth)):
        if computed[i] == positive_label:
            if truth[i] == positive_label:
                TP += 1
            else:
                FP += 1
        else:
            if truth[i] == positive_label:
                FN += 1
            else:
                TN += 1

    return TP,TN,FP,FN

def getAccuracy(TP,TN,FP,FN):
    if TP+TN+FP+FN == 0:
        return 0
    return (TP + TN) / (TP+TN+FP+FN)

def getPrecision(TP,TN,FP,FN):
    if TP + FP == 0:
        return 0
    return TP/(TP + FP)

def getRecall(TP,TN,FP,FN):
    if TP+FN == 0:
        return 0
    return TP/(TP+FN)

In [19]:
import pandas as pd
from PIL import Image

def read_datas() -> pd.DataFrame:
    df = pd.read_csv('datas.csv')
    df = df.dropna()

    return df

def getTrainingAndValidationDatas():
    np.random.seed(5)
    df = read_datas()
    n = df.shape[0]
    indexes = [i for i in range(n)]
    trainingIndexes = np.random.choice(indexes, int(0.7 * n), replace = False)
    validationIndexes = [i for i in range(n) if not i in trainingIndexes]

    trainingInputs = [df['Photo'].iloc[i] for i in trainingIndexes]
    trainingOutputs = [df['Has Filter'].iloc[i] for i in trainingIndexes]

    validationInputs = [df['Photo'].iloc[i] for i in validationIndexes]
    validationOutputs = [df['Has Filter'].iloc[i] for i in validationIndexes]

    return trainingInputs, trainingOutputs, validationInputs, validationOutputs

def getInputParameters(inputImages, size):
    params = []
    for imagePath in inputImages:
        params.append([])
        image = Image.open(imagePath)
        image = image.resize(size)
        width, height = image.size
        pixel_data = list(image.getdata())
        for y in range(height):
            row_pixels = pixel_data[y * width : (y + 1) * width]
            params[-1].append([])
            for pixel in row_pixels:
                r,g,b = pixel[0],pixel[1],pixel[2]
                maxi = max([r,g,b])
                mini = min([r,g,b])
                if mini == maxi:
                    r = g = b = 0
                else:
                    r = (r-mini)/(maxi-mini)
                    g = (g-mini)/(maxi-mini)
                    b = (b-mini)/(maxi-mini)
                params[-1][-1].append([r,g,b])
    return params

def tang_func(x):
    from numpy import tanh
    return tanh(x)

def getClassifier():
    trainingInputs, trainingOutputs, validationInputs, validationOutputs = getTrainingAndValidationDatas()
    x_train = getInputParameters(trainingInputs,(62,62))
    x_valid = getInputParameters(validationInputs,(62,62))
    cnn = ConvNeuralNetwork()
    cnn.add_conv_layer(3,(3,3))
    cnn.add_maxpool_layer((4,4))
    cnn.add_fully_connected_layer(input_size=15*15*9,ouput_size=10, activation_function=tang_func)
    cnn.add_fully_connected_layer(input_size=10,ouput_size=2)
    cnn.train(x_train, trainingOutputs,5)

    outut = cnn.predict(x_valid)

    TP,TN,FP,FN = get_TP_TN_FP_FN(validationOutputs, outut,'YES')
    accuracy = getAccuracy(TP,TN,FP,FN)
    precision = getPrecision(TP,TN,FP,FN)
    recall = getRecall(TP,TN,FP,FN)
    print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}")
    
    return cnn

getClassifier()

Epoch 0 -------> loss: 2.311518761902344
Epoch 1 -------> loss: 2.3115189171068695
Epoch 2 -------> loss: 2.3115190723101824
Epoch 3 -------> loss: 2.311519227512095
Epoch 4 -------> loss: 2.3115193827128677
Accuracy: 0.37037037037037035
Precision: 0.32
Recall: 1.0
